In [ ]:
#installation stuff
!pip install qiskit #2.0.0 let's try
!pip install qiskit-aer
import qiskit
print(qiskit.__version__)
from math import pi
!pip install --upgrade pip
!pip install qiskit==1.4.3
!pip install qiskit-terra qiskit-aer qiskit-visualization
!pip install numpy matplotlib

In [ ]:
#to-do time stuff
import time
start = time.time() #before exec
end = time.time() #after result
total_time += (end - start)
print(f"time elapsed {total_time}s")

In [ ]:
#to-do benchmarking stuff
t_num_qubits = qc.num_qubits
t_num_gates = qc.size()
print(f"circuit consists of {t_num_qubits} qubits, and {t_num_gates} gates")

In [ ]:
#import stuff
from qiskit.primitives import Sampler
from qiskit_aer import AerSimulator
import random
import math
from qiskit import QuantumCircuit
import matplotlib.pyplot as plt
import numpy as np
import random
import math
from qiskit import transpile
from qiskit.visualization import plot_histogram
semiprimes = {5: 21}
results = {}

In [ ]:
#iqft
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    return qc

In [ ]:
#amod
def amodN(a, power, N, n_count):
    U = QuantumCircuit(n_count)
    for i in range(n_count):
        U.h(i)
    for j in range(n_count):
        a_mod_exp = pow(a, 2**j, N)
        phase_value = 2 * np.pi * a_mod_exp / N
        for q in range(n_count):
            U.p(phase_value, q)
    U = U.to_gate()
    c_U = U.control()
    return c_U

In [ ]:
#main part
for index, (_, N) in enumerate(semiprimes.items()):
    total_time = 0
    times = 0
    used_a = []
    while True:
        times += 1
        p_f, q_f = 1, 1
        a = random.randint(2, N-2)
        if a in used_a:
            break
        print(f"for a = {a}")
        if math.gcd(a, N) > 1:
            p_f = math.gcd(a, N)
            q_f = N // p_f
            break

        n_qubits = (N.bit_length()  // 2 ) - 1
        qc = QuantumCircuit(2 * n_qubits, n_qubits)
        for q in range(n_qubits):
            qc.h(q)
        qc.x(n_qubits * 2 - 1)

        for q in range(n_qubits):
            qc.append(amodN(a, 2**q, N, n_qubits), [q] + [i + n_qubits for i in range(n_qubits)])
        qc.append(qft_dagger(n_qubits), range(n_qubits))
        qc.measure(range(n_qubits), range(n_qubits))

        #simulations
        shots = 1024
        mybackend = AerSimulator()
        qc_transpiled = transpile(qc, mybackend, initial_layout=[i for i in range(qc.num_qubits)])
        job = mybackend.run(qc_transpiled, shots=shots)
        result = job.result()
        counts = result.get_counts()

        r = int(max(counts, key=counts.get), 2)
        factor1 = math.gcd(a**(r//2) - 1, N)
        factor2 = math.gcd(a**(r//2) + 1, N)
        if factor1 == 1 and factor2 > 1:
            p_f = factor2
            q_f = N // factor2
        elif factor1 > 1 and factor2 == 1:
            p_f = factor1
            q_f = N // factor1

        if p_f != 1 and q_f != 1 and p_f * q_f == N:
            break

        used_a.append(a)

    print(f"Factorized {N} into {p_f}, {q_f}")
    print("*" * 10)
    results[N] = [(p_f, q_f), times, total_time]

In [ ]:
#visualize
plot_histogram(counts)

In [ ]:
#clean up
del q, c, qc
del result
del job

In [ ]:
#ayo's code
def qft(qc, num_qubits):
  for i in range(num_qubits - 1):
    qc.h(i)
    for k in range(2 + i, num_qubits + 1):
      qc.cp(2 * pi / (2 ** k), k - 1, i)
  qc.h(num_qubits - 1)

  for j in range(num_qubits // 2):
    qc.swap(j, num_qubits - 1 - j)

for i in range(m):
  qc.h(i)
qc.x(m)
qc.h(m)

for j in range(m - 1, -1, -1):
  for k in range(2 ** (m - 1 - j)):
    qc.cx(j, m)
# Inverse QFT part
iqft(qc, m)

for i in range(m):
  qc.measure(i, i)